In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [49]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage, HumanMessage

In [22]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [23]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)

docs = loader.load()

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [27]:
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ]
)

history_aware_retriever = create_history_aware_retriever(llm, retriever, contextualize_q_prompt)

In [28]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [30]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [36]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [37]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },
)["answer"]

'Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable and understandable. Techniques like Chain of Thought (CoT) and Tree of Thoughts are used to decompose hard tasks into multiple manageable tasks by instructing models to think step by step or exploring multiple reasoning possibilities at each step. Task decomposition can be achieved through simple prompting by LLM, task-specific instructions, or human inputs.'

In [39]:
conversational_rag_chain.invoke(
    {"input": "What are the common ways of doing it?"},
    config={
        "configurable": {"session_id":"abc123"}
    }
)["answer"]

'Task decomposition can be done in common ways such as using prompting techniques like Chain of Thought (CoT) or Tree of Thoughts to break down complex tasks into smaller steps. This can involve instructing models to "think step by step" or exploring multiple reasoning possibilities at each step. Additionally, task decomposition can also be achieved through task-specific instructions or human inputs to provide guidance on how to break down the task effectively.'

In [50]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"
    
    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition involves breaking down a complex task into smaller and simpler steps to make it more manageable and understandable. Techniques like Chain of Thought (CoT) and Tree of Thoughts are used to decompose hard tasks into multiple manageable tasks by instructing models to think step by step or exploring multiple reasoning possibilities at each step. Task decomposition can be achieved through simple prompting by LLM, task-specific instructions, or human inputs.

User: What are the common ways of doing it?

AI: Task decomposition can be done in common ways such as using prompting techniques like Chain of Thought (CoT) or Tree of Thoughts to break down complex tasks into smaller steps. This can involve instructing models to "think step by step" or exploring multiple reasoning possibilities at each step. Additionally, task decomposition can also be achieved through task-specific instructions or human inputs to provide guidance on how to bre